# Imports, Globals, and Loading Dataframes

In [ ]:
#import management
import autoreload
%load_ext autoreload
%autoreload 2  # Autoreload all modules

import importlib

#standard imports
import sys
import os
from datetime import datetime
import pickle

#custom modules
#import localSettings as ls
import plotter
import NUMUhelper as moreFunctions

#scientific imports
import uproot
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.pylab as pylab
import matplotlib.backends.backend_pdf
import numpy as np
import math

#put these throughout the code to reload if needed
importlib.reload(ls)
importlib.reload(plotter)
importlib.reload(moreFunctions)

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

In [ ]:
#############################################
# SETUP LOCAL SESSION ENVIRONMENT 
main_path = ls.main_path
sys.path.append(main_path)
now = datetime.now()
date_time = now.strftime("%m%d%Y")
print("date and time:",date_time)

#############################################
# WHAT KIND OF DATA ARE WE LOOKING AT TODAY?

RUN = 3 # 1, 2, 3
PRESEL = True # load preselected (slimmed) data?

MC_SAMPLE_TYPE = '0702' #0304, 0702, ...
BNB_SAMPLE_TYPE = '0617' #0304, 0617, fake_data, ...

if BNB_SAMPLE_TYPE == 'fake_data': FAKEDATASET = 'set1' #which fake data set?

LOADDETSYS = True # Utilize MC systematic errors?
USECRT = False # Load and apply CRT cuts?
FLATTEN = False # plotter.py expects this to be false
NPROTON_CAT = True # break 1muNp into individual proton multiplicites?

## Load Dataframes

In [ ]:
#####################################################
# Setting datapaths here instead of localsettings.py
# Adds a bit of flexibility

tree = "NeutrinoSelectionFilter"
##################### SET DIRECTORY PATHS #####################
if PRESEL:
    MC_PATH = "E:\\HEPPA\\Data\\PeLEE\\{}_numupresel\\Run{}\\".format(MC_SAMPLE_TYPE, RUN)
    BNB_PATH = "E:\\HEPPA\\Data\\PeLEE\\{}_numupresel\\Run{}\\".format(BNB_SAMPLE_TYPE, RUN)
    APPEND = "_numupresel"
else:
    MC_PATH = "E:\\HEPPA\\Data\\PeLEE\\{}\\Run{}\\".format(MC_SAMPLE_TYPE, RUN)
    BNB_PATH = "E:\\HEPPA\\Data\\PeLEE\\{}\\Run{}\\".format(BNB_SAMPLE_TYPE, RUN)
    APPEND = ""
    
##################### SET FILE NAMES ##############################
if RUN == 3:
    ######## MC + EXT ##############
    if MC_SAMPLE_TYPE == '0304':
        EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_G1_all_reco2'+APPEND
        #EXT = 'ext'+ls.APPEND
        NU  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run3_reco2_G_reco2'+APPEND
        DRT = 'prodgenie_bnb_dirt_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2'+APPEND
    elif MC_SAMPLE_TYPE == '0702':
        EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_G1_all_reco2'
        NU = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run3_reco2_G_reco2'
        DRT = 'prodgenie_bnb_dirt_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2'
    ######## BNB ###############
    if BNB_SAMPLE_TYPE == '0304':
        BNB = 'neutrinoselection_filt_G1'+APPEND
    elif BNB_SAMPLE_TYPE == 'fake_data':
        BNB = 'prod_uboone_nu2020_fakedata_{}_run3b_reco2_v08_00_00_41_reco2'.format(FAKEDATASET)
    elif BNB_SAMPLE_TYPE == '0617':
        BNB = 'data_bnb_peleeFilter_uboone_v08_00_00_41_pot_run3_G1_neutrinoselection_filt'
    else:
        print("{} is not a known sample type".format(SAMPLE_TYPE))

    
mc = uproot.open(MC_PATH+NU+".root")[ls.fold][tree]
ext = uproot.open(MC_PATH+EXT+".root")[ls.fold][tree]
dirt = uproot.open(MC_PATH+DRT+".root")[ls.fold][tree]
data = uproot.open(BNB_PATH+BNB+".root")[ls.fold][tree]

uproot_v = [mc,ext,data,dirt]

#basically all the variables that are used for this selection
variables = [
    "nslice", 
    "interaction", "nu_pdg", "category", 
    "slpdg", "trk_score_v", "backtracked_pdg", 
    #'theta',
    "topological_score",
    "trk_energy_tot", "ccnc",
    #"trk_bkt_pdg", "hits_ratio", "n_tracks_contained", 
    "NeutrinoEnergy2",
    "run","sub","evt",
    "trk_llr_pid_score_v", # trk-PID score
    "trk_energy_proton_v", # track energy under proton hyp
    "trk_energy_muon_v", # track energy under muon hyp
    #'true_nu_vtx_x','true_nu_vtx_y','true_nu_vtx_z',
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "nproton",# "nmuon",
    "nu_e", "trk_distance",
    "trk_phi", "trk_theta",
    'slnunhits',"slnhits",
    #"mc_pdg", "shr_score", 
    #"npi0","npion","pion_e","muon_e","true_e_visible",
    #"slclustfrac", "reco_nu_vtx_x", "reco_nu_vtx_y", "reco_nu_vtx_z","contained_fraction",
    "trk_sce_start_x_v","trk_sce_start_y_v","trk_sce_start_z_v",
    "trk_sce_end_x_v","trk_sce_end_y_v","trk_sce_end_z_v",
    "trk_mcs_muon_mom_v","trk_range_muon_mom_v", "trk_len_v",
    "pfp_generation_v","trk_distance_v","trk_theta_v","trk_phi_v",
    #"trk_energy_muon","trk_energy_tot","trk_energy",
    "pfnhits","pfnunhits",
]

#some of these files are picky eaters
slimmed_variables = [
    "nslice", 
    "selected", "nu_pdg",
    'nproton',
    "slpdg", "trk_score_v","slclustfrac",
    #"contained_fraction",
    "backtracked_pdg","category",
    "topological_score",
    "run", "sub", "evt",
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "trk_sce_start_x_v","trk_sce_start_y_v","trk_sce_start_z_v",
    "trk_sce_end_x_v","trk_sce_end_y_v","trk_sce_end_z_v",
    "trk_mcs_muon_mom_v","trk_range_muon_mom_v", "trk_len_v",
    'trk_llr_pid_score_v',
    "pfp_generation_v","trk_distance_v","trk_theta_v","trk_phi_v",
    #"trk_energy_muon","trk_energy_tot","trk_energy",
    'trk_energy_muon_v','trk_energy_proton_v',
    "pfnhits","pfnunhits",
    'slnunhits','slnhits',
    'NeutrinoEnergy2',
]

#just the variables necessary for the preselection,
presel_input = [
    'nslice', 'run',
    #'topological_score',
    #'reco_nu_vtx_sce_x','reco_nu_vtx_sce_y','reco_nu_vtx_sce_z',
    'pfnunhits', 'pfnhits','trk_score_v',
]
mc_presel_input = [
    'category', '_opfilter_pe_beam', '_opfilter_pe_veto',
    'nu_pdg', 'slnunhits', 'slnhits', 'nproton',
    'ccnc','contained_fraction'
]

if USECRT:
    variables.append("_closestNuCosmicDist")
    variables.append("crtveto")
    variables.append("crthitpe")
    variables.append("CosmicIP")
    slimmed_variables.append("_closestNuCosmicDist")
    slimmed_variables.append("crtveto")
    slimmed_variables.append("crthitpe")
    slimmed_variables.append("CosmicIP")
    
#sets remove redundancies  
slimmed_variables = list(set(slimmed_variables))
variables = list(set(variables))
print(variables)
    
WEIGHTS = ["weightSpline","weightTune","weightSplineTimesTune"]
if LOADDETSYS: WEIGHTS += ["weightsGenie", "weightsFlux", "weightsReint"]

MCVARS = [
    "_opfilter_pe_beam", "_opfilter_pe_veto", # did the event pass the common optical filter (for MC only)
]


print("starting to build dataframes...")
data = data.pandas.df(slimmed_variables, flatten=FLATTEN)
ext = ext.pandas.df(slimmed_variables, flatten=FLATTEN)
print("Data and EXT dataframes built...")
if PRESEL:
    presel,_ = moreFunctions.get_NUMU_sel(USECRT, opfilter=False)
    data = data.query(presel)
    ext = ext.query(presel)    
mc = mc.pandas.df(variables + WEIGHTS + MCVARS, flatten=FLATTEN)
print("MC dataframe built...")
dirt = dirt.pandas.df(variables + WEIGHTS + MCVARS, flatten=FLATTEN)
print("Dirt dataframe built...")
if PRESEL:
    presel,_ = moreFunctions.get_NUMU_sel(USECRT, opfilter=True)
    mc = mc.query(presel)
    dirt = dirt.query(presel)

In [ ]:
#Calculated columns and various corrections
df_v = [mc,ext,data,dirt]
names_v = ['mc','ext','data','dirt']

M_mu = 0.105 #GeV/c
M_p = 0.938 #GeV/c
M_n = 0.939 #GeV/c
B = 0.04 #binding energy of argon used in simulation
proton_pidscore = -0.2


for name,df in zip(names_v,df_v):
    print(name)
    #useful, but not necessary calculated variables
    df['trk_p_quality_v'] = (df['trk_mcs_muon_mom_v']-df['trk_range_muon_mom_v'])/df['trk_range_muon_mom_v']
    df['trk_cos_theta_v'] = df['trk_theta_v'].apply(lambda x: np.cos(x))
    df['trk_sin_theta_v'] = df['trk_theta_v'].apply(lambda x: np.sin(x))
    df['trk_cos_phi_v'] = df['trk_phi_v'].apply(lambda x: np.cos(x))
    df['trk_sin_phi_v'] = df['trk_phi_v'].apply(lambda x: np.sin(x))
    df['trk_range_proton_mom_v'] = df['trk_energy_proton_v'].apply(lambda x: np.sqrt(2*M_p*x))
    df['trk_range_muon_e_v'] = (df['trk_range_muon_mom_v']**2 + M_mu**2)**.5 # E
    df['trk_range_muon_ke_v'] = df['trk_range_muon_e_v'] - M_mu #KE
    df['trk_energy_tot'] = df["trk_energy_proton_v"].apply(lambda x: sum(x)) #is missing from G1 sample
    df['reco_nu_e_range_v'] = df["trk_range_muon_e_v"] + (df["trk_energy_tot"] - df["trk_energy_proton_v"])     
    #protons have trk_score cut and llr_pid_score cut
    proton_mask = df['trk_score_v'].apply(lambda x: x>0.5) * df['trk_llr_pid_score_v'].apply(lambda x: x<proton_pidscore)
    df['reco_nproton'] = (df['trk_llr_pid_score_v']*proton_mask).apply(lambda x: len(x[x!=False]))
    df['reco_ntrack'] = df['trk_score_v'].apply(lambda x: len(x))
    # break momentum vector apart
    df['trk_dx_v'] = df['trk_sin_theta_v']*df['trk_cos_phi_v']
    df['trk_dy_v'] = df['trk_sin_theta_v']*df['trk_sin_phi_v']
    df['trk_dz_v'] = df['trk_cos_theta_v']

    df["slclnhits"] = df["pfnhits"].apply(lambda x: sum(x))
    df["slclnunhits"] = df["pfnunhits"].apply(lambda x: sum(x))
    
    #just MC stuff
    if name in ['mc','dirt']:
        df['backtracked_pdg_v'] = df['backtracked_pdg']
        df.loc[ df['weightTune'] <= 0, 'weightTune' ] = 1.
        df.loc[ df['weightTune'] == np.inf, 'weightTune' ] = 1.
        df.loc[ df['weightTune'] > 100, 'weightTune' ] = 1.
        df.loc[ np.isnan(df['weightTune']) == True, 'weightTune' ] = 1.
        df.loc[ df['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
        df.loc[ df['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
        df.loc[ df['weightSplineTimesTune'] > 100, 'weightSplineTimesTune' ] = 1.
        df.loc[ np.isnan(df['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1.
    if name == 'mc':
        df.loc[(df['category']!=1)&(df['category']!=10)&(df['category']!=11)&(df['category']!=111)&(df['slnunhits']/df['slnhits']<0.2), 'category'] = 4
        if NPROTON_CAT:
            df.loc[(df['category']==2)&(df['nproton']==0), 'category'] = 22
            df.loc[(df['category']==2)&(df['nproton']==1), 'category'] = 23
            df.loc[(df['category']==2)&(df['nproton']==2), 'category'] = 24
            df.loc[(df['category']==2)&(df['nproton']>=3), 'category'] = 25
        
    #This information is useful for applying corrections and such
    if name == 'data': df['bnbdata'] = 1
    else: df['bnbdata'] = 0
    if name == 'ext': df['extdata'] = 1
    else: df['extdata'] = 0
    if name in ['ext','data']:
        #column needs to exist, even if not cut on 
        df['_opfilter_pe_beam'] = 999
        df['_opfilter_pe_veto'] = -999
    if i not in ['mc','dirt'] and USECRT:
        #only apply to data and ext
        df.loc[(df['run'] > 16300),'crthitpe'] = df['crthitpe']*1.09 #hitpe correction
        
#break mc sample into numu and nue using truth info
nue = mc.query('nu_pdg == 12 or nu_pdg == -12')
mc  = mc.query('nu_pdg == 14 or nu_pdg == -14')    

# Beginning of analysis section
## organize samples (for easy switching)
## Setup, define useful functions

In [ ]:
importlib.reload(moreFunctions)
#put the samples in a way that's convient to acces later

mc_type = MC_SAMPLE_TYPE
if BNB_SAMPLE_TYPE == 'fake_data':
    bnb_type = 'FD' + FAKEDATASET[-1]
else:
    bnb_type = BNB_SAMPLE_TYPE
    
weights, pot = moreFunctions.get_scaling(RUN, bnb_type, mc_type)

samples = {"mc": mc,"nue": nue,"data": data,"ext": ext,"dirt": dirt}

# ensure presel consistnecy
if PRESEL:
    for sample in samples:
        if sample in ['mc','nue','dirt']: OPFIL = True
        else: OPFIL = False
        presel,_ = moreFunctions.get_NUMU_sel(USECRT, opfilter=OPFIL)
        samples[sample] = samples[sample].query(presel)#.dropna()
    
my_plotter = plotter.Plotter(samples, weights, pot=pot)

print("weights:")
for weight in weights:
    print("{}: {}".format(weight,weights[weight]))
print("POT: {}".format(pot))

In [ ]:
importlib.reload(moreFunctions)
get_NUMU_sel = moreFunctions.get_NUMU_sel
update_data = moreFunctions.update_data
Eff = moreFunctions.Eff
Pur = moreFunctions.Pur
apply_fullsel = moreFunctions.apply_fullsel
apply_fullsel_noMCS = moreFunctions.apply_fullsel_noMCS
get_presel_index = moreFunctions.get_presel_index

### Apply Selections

In [ ]:
importlib.reload(moreFunctions)
fullsel_samples = {}
presel_samples = {}
query,_ = moreFunctions.get_NUMU_sel(USECRT)

for sample in samples:
    presel_samples[sample] = samples[sample].query(query)
    if "presel" in ls.SAMPLE:
        samples[sample] = presel_samples[sample]
        
for sample in samples:
    print("{}: {}".format(sample, moreFunctions.get_current_time("%H:%M:%S")))
    fullsel_samples[sample] = moreFunctions.apply_muon_fullsel(presel_samples[sample], sample, USECRT, LOADDETSYS)
    fullsel_samples[sample]['reco_ntrack'] = presel_samples[sample].loc[fullsel_samples[sample].index]['reco_ntrack']
    
#turn a few of these vector-valued Series into scalar-values ones (because they should be after the full selection)
VECS = [
    'reco_nu_e_range_v', 'trk_range_muon_e_v',
    'trk_len_v','trk_cos_theta_v'
]
for sample in fullsel_samples:
    for vec in VECS:
        fullsel_samples[sample][vec[:-2]] = fullsel_samples[sample][vec].apply(lambda x: x[0])

In [ ]:
importlib.reload(moreFunctions)
CCQE_presel_samples = {}
CCQE_muon_samples = {}
CCQE_proton_samples = {}
CCQE_samples = {}
verbose = False
for sample in samples:
    print(sample)
    #events that have atleast one muon candidate and exactly 2 reco tracks
    if verbose: print("presel...")
    CCQE_presel_samples[sample] = moreFunctions.apply_CCQE_presel(samples[sample], sample, USECRT, LOADDETSYS)
    if verbose: print("muons...")
    CCQE_muon_samples[sample] = moreFunctions.select_muons(CCQE_presel_samples[sample], sample, USECRT, LOADDETSYS)
    if verbose: print("protons...")
    CCQE_proton_samples[sample] = moreFunctions.select_protons(CCQE_presel_samples[sample], sample, USECRT, LOADDETSYS)
    if verbose: print("done")
    #intersection of muon & proton dataframes are the 1mu1p events
    muon_entries = set(CCQE_muon_samples[sample].index)
    proton_entries = set(CCQE_proton_samples[sample].index)
    intersec = list(muon_entries & proton_entries)
    #update everything
    CCQE_samples[sample] = CCQE_presel_samples[sample].loc[intersec]
    CCQE_muon_samples[sample] = CCQE_muon_samples[sample].loc[intersec]
    CCQE_proton_samples[sample] = CCQE_proton_samples[sample].loc[intersec]
    #make sure this is fixed
    if sample in ['mc','dirt','nue']:
        CCQE_samples[sample]['backtracked_pdg_v'] = CCQE_samples[sample]['backtracked_pdg_v'].apply(lambda x: [int(round(xx)) for xx in x])
        #CCQE_proton_samples[sample]['backtracked_pdg_v'] = CCQE_proton_samples[sample]['backtracked_pdg_v'].apply(lambda x: [int(round(xx)) for xx in x])
        #CCQE_muon_samples[sample]['backtracked_pdg_v'] = CCQE_muon_samples[sample]['backtracked_pdg_v'].apply(lambda x: [int(round(xx)) for xx in x])
        
#make the CCQE calculated variables
M_mu = 0.105 #GeV/c
M_p = 0.938 #GeV/c
M_n = 0.939 #GeV/c
B = 0.04 #binding energy of argon used in simulation

for sample in CCQE_samples:
    df = CCQE_samples[sample]
    df_muon = CCQE_muon_samples[sample]
    df_proton = CCQE_proton_samples[sample]
    ########
    df_muon['p'] = df_muon['trk_range_muon_mom_v']
    df_muon['E'] = df_muon['trk_range_muon_e_v']
    df_muon['px'] = df_muon['p']*df_muon['trk_dx_v']
    df_muon['py'] = df_muon['p']*df_muon['trk_dy_v']
    df_muon['pz'] = df_muon['p']*df_muon['trk_dz_v']
    df_muon['pT'] = (df_muon['px']**2 + df_muon['py']**2)**0.5
    df_muon['pTransverseRatio'] = df_muon['pT'] / df_muon['p']
    df_muon['trk_len'] = df_muon['trk_len_v'].apply(lambda x: x[0])
    ########
    df_proton['p'] = df_proton['trk_range_proton_mom_v']
    df_proton['E'] = df_proton['trk_energy_proton_v']
    df_proton['px'] = df_proton['p']*df_proton['trk_dx_v']
    df_proton['py'] = df_proton['p']*df_proton['trk_dy_v']
    df_proton['pz'] = df_proton['p']*df_proton['trk_dz_v']
    df_proton['pT'] = (df_proton['px']**2 + df_proton['py']**2)**0.5
    df_proton['pTransverseRatio'] = df_proton['pT'] / df_proton['p']
    df_proton['trk_len'] = df_proton['trk_len_v'].apply(lambda x: x[0])
    #########
    df['Q2'] = 2 * df_muon['reco_nu_e_range_v'] * (df_muon['trk_range_muon_e_v'] - df_muon['p']**2) - M_mu**2
    df['Mhad'] = df_muon['reco_nu_e_range_v'] - df_muon['trk_range_muon_e_v']
    df['Xbj'] = df['Q2'] / (2 * M_n * df['Mhad'])
    df['Ybj'] = df['Mhad'] / df_muon['reco_nu_e_range_v']
    df['pT'] = ((df_muon['px'] + df_proton['px'])**2 + (df_muon['py'] + df_proton['py'])**2)**0.5
    df['p'] = ((df_muon['px'] + df_proton['px'])**2 + (df_muon['py'] + df_proton['py'])**2 + (df_muon['pz'] + df_proton['pz'])**2)**0.5
    df['pL'] = df_proton['pz']**2 + df_muon['pz']**2
    df['pTransverseRatio'] = df['pT'] / df['p']
    #opening angle
    # cos(opening angle) = dot(muon unit vector, proton unit vector)
    df['cos_alpha'] = df_muon['trk_dx_v']*df_proton['trk_dx_v'] + df_muon['trk_dy_v']*df_proton['trk_dy_v'] + df_muon['trk_dz_v']*df_proton['trk_dz_v']
    df['opening_angle'] = df['cos_alpha'].apply(lambda x: np.arccos(x))
    df['phi_diff'] = df_muon['trk_phi_v'] - df_proton['trk_phi_v']
    df['theta_tot'] = df_muon['trk_theta_v'] + df_proton['trk_theta_v']
    longest_mask = df['trk_len_v'].apply(lambda x: x == x[list(x).index(max(x))])#identify longest
    df['trk_len'] = (df['trk_len_v']*longest_mask).apply(lambda x: x[x!=False]) #longest available track

# Export Data Files for Constraint

In [ ]:
#select the longest track for the dataframes
#only need to do this for variables of interest
variables = ['trk_theta_v','trk_range_muon_e_v','reco_nu_e_range_v']

for sample in fullsel_samples.keys():
    df = fullsel_samples[sample]
    trk_lens = df['trk_len_v']
    longest_mask = trk_lens.apply(lambda x: x == x[list(x).index(max(x))]) #identify longest
    for variable in variables:
        VAR = df[variable]
        VAR = VAR.apply(lambda x: x[~np.isnan(x)]) #clean up nan vals
        VAR = VAR[VAR.apply(lambda x: len(x) > 0)]
        VAR = (VAR*longest_mask).apply(lambda x: x[x != False]) #apply longest mask
        if len(VAR.iloc[0]) == 1:
            VAR = VAR.apply(lambda x: x[0])
            #apply this new column
            new_variable = variable[:-2]
            df[new_variable] = VAR
        else:
            print('something is wrong...')
            print(VAR)
    

In [ ]:
# want to spit out the bin counts for full selection
# just copy and paste output to give to David/Maya

binedges = np.linspace(0.15,1.55,15)
print("binedges:{}".format(binedges))

for s,sample in enumerate(fullsel_samples):
    print(sample)
    df = fullsel_samples[sample].copy()
    binvals,_ = np.histogram(df['reco_nu_e_range'], bins=binedges)
    print(binvals," -> ",sum(binvals))
        

In [ ]:
# get the events that pass the selection cuts
# and save info about the event into this txt file

for sample in ['mc','data','dirt','ext']:
    print(sample)
    if sample in ['mc','dirt','ext']:
        SAVEPATH = "passing_events\\{}\\".format(mc_type)
        if not os.path.exists(SAVEPATH): os.makedirs(SAVEPATH)
        f_out = open(SAVEPATH+"{}_Passingevents_{}_{}.txt".format(sample,mc_type,date_time),'w')
    else:
        SAVEPATH = "passing_events\\{}\\".format(bnb_type)
        if not os.path.exists(SAVEPATH): os.makedirs(SAVEPATH)
        f_out = open(SAVEPATH+"{}_Passingevents_{}_{}.txt".format(sample,bnb_type,date_time),'w')
    #filter out poorly reconstructed things...maybe deal with this upstream a bit better?
    df = fullsel_samples[sample].query('reco_nu_e_range >= 0')
    
    print ('file %s_final has %i selected entries'%(sample,df.shape[0]))
    f_out.write('%s\t%s\t%s\t%s\t%s\t%s\n'%('run','sub','evt','angle','Emuon','Erange'))
    for i,row in df.iterrows():
        run = row['run']
        sub = row['sub']
        evt = row['evt']
        angle = row['trk_theta']
        Emuon = row['trk_range_muon_e']
        Erange = row['reco_nu_e_range']
        f_out.write('%i\t%i\t%i\t%.4f\t%.4f\t%.4f\n'%(run,sub,evt,angle,Emuon,Erange))
    f_out.close()

# Plotting

In [ ]:
############################################
## Make just one plot with or without ratio plot
############################################
importlib.reload(moreFunctions)

#active volume
AVx = [-1.55,254.8]
AVy = [-115.53, 117.47]
AVz = [0.1, 1036.9]

VARIABLE, BINS, RANGE, XTIT = 'nu_e',14,(0.15,1.55),r'True $\nu$ Energy [GeV]'
VARIABLE, BINS, RANGE, XTIT = 'reco_ntrack',5,(0.5,5.5),r'Reco Track Multiplicity'
#VARIABLE, BINS, RANGE, XTIT = 'nproton',5,(-0.5,4.5),r'True Proton Multiplicity'
#VARIABLE, BINS, RANGE, XTIT = 'trk_len_v',24,(0,100),r'Track Length [cm]'
#VARIABLE, BINS, RANGE, XTIT = 'trk_score_v',24,(0.5,1),r'Track Score'
#VARIABLE, BINS, RANGE, XTIT = 'trk_distance_v',24,(0,10),r'Track Distance [cm]'
#VARIABLE, BINS, RANGE, XTIT = 'trk_p_quality_v',24,(-1,2.5),r'MCS Consistency $(\frac{P_{MCS}-P_{Range}}{P_{Range}})$'
#VARIABLE, BINS, RANGE, XTIT = 'crtveto',2,(-0.5,1.5),r'CRT Veto'
VARIABLE, BINS, RANGE, XTIT = '_closestNuCosmicDist', 25, (0,100), r'Closest $\nu$-Cosmic Distance [cm]'
#VARIABLE, BINS, RANGE, XTIT = 'trk_llr_pid_score_v', 25, (-1, 1), "LLR PID Score"
#VARIABLE, BINS, RANGE, XTIT = 'Q2_v', 25, (0,2), "Q2"
#VARIABLE, BINS, RANGE, XTIT = 'Mhad_v', 25, (0,2.5), r"M$_{hadron}$"
#VARIABLE, BINS, RANGE, XTIT = 'Xbj_v', 25, (0,3), r"Bjorken x"
#VARIABLE, BINS, RANGE, XTIT = 'Ybj_v', 25, (0,2), r"Bjorken y"
VARIABLE, BINS, RANGE, XTIT = 'trk_cos_theta_v',24,(-1,1),r'Cos($\theta$)'
#VARIABLE, BINS, RANGE, XTIT = 'trk_energy_proton_v', 25, (0,1), r'Reco Range-Based Proton Energy [GeV]'
#VARIABLE, BINS, RANGE, XTIT = 'NeutrinoEnergy2', 25, (0,2000), 'Visible Energy on Plane 2 [MeV]'
#VARIABLE, BINS, RANGE, XTIT = 'trk_range_muon_e_v', 10, (0.15,1.15), r'Reco Range-Based Muon Energy [GeV]'
#VARIABLE, BINS, RANGE, XTIT = 'reco_nu_e_range_v',14,(0.15,1.55),r'Reco Range-Based $\nu$ Energy [GeV]'
#VARIABLE, BINS, RANGE, XTIT = 'contained_fraction',25,(0,1),r'Contained Fraction'
##########################
# VARIABLES with detsys available
#VARIABLE, BINS, RANGE, XTIT = 'trk_len_v',24,(0,500),r'Track Length [cm]'
#VARIABLE, BINS, RANGE, XTIT = 'trk_cos_v',24,(-1,1),r'Cos($\theta$)'
#VARIABLE, BINS, RANGE, XTIT = 'reco_nu_e_range_v',14,(0.15,1.55),r'Reco Range-Based $\nu$ Energy [GeV]'
#VARIABLE, BINS, RANGE, XTIT = 'trk_range_muon_e_v',10,(0.15,1.15),r'Reco Range-Based Muon Energy [GeV]'
#VARIABLE, BINS, RANGE, XTIT = 'trk_llr_pid_score_v',24,(-1,1), 'Log-Likelihood PID Score'

##########################
# USE get_plots TO MAKE BATCHES OF PLOTS
VARIABLES, BIN, RANGES, XTITS = [VARIABLE], [BINS], [RANGE], [XTIT]
#VARIABLES, BIN, RANGES, XTITS = moreFunctions.get_plots('muon input')
#VARIABLES, BIN, RANGES, XTITS = moreFunctions.get_plots('fullselKinematics')
#VARIABLES, BIN, RANGES, XTITS = moreFunctions.get_plots('presel input')
#VARIABLES, BIN, RANGES, XTITS = moreFunctions.get_plots('crt vars')
#VARIABLES, BIN, RANGES, XTITS = moreFunctions.get_plots('CCQE')
#VARIABLES, BIN, RANGES, XTITS = moreFunctions.get_plots('CCQE_proton_kinematics')
#VARIABLES, BIN, RANGES, XTITS = moreFunctions.get_plots('CCQE_muon_kinematics')

####################################################################
## MAIN CONTROL AREA
#tag gets added to end of .pdf name and changes some parameters
tag = "fullsel_samples"
samples = fullsel_samples #samples, presel_samples, fullsel_samples
SAVEFIG = True
DRAWDATA = True
DRAWRATIO = True
SELECTLONGEST = True
DRAWSYS = True #need to load extra branches (weightsFlux, weightsGenie, weightsReint)
DETSYS = True
KINDS = ['event_category','interaction'] #interaction, backtracked_pdg, event_category
KINDS = ['event_category']

if DETSYS == False and DRAWSYS == True:
    SAVEDIR = 'C:\\Users\\Ryan\\python-workspace\\PELEE-newmcfilteredsamples\\plots\\{}-{}\\Run{}\\StatsandFluxGenieReint\\'.format(bnb_type,mc_type,RUN)
elif DETSYS and DRAWSYS:
    SAVEDIR = 'C:\\Users\\Ryan\\python-workspace\\PELEE-newmcfilteredsamples\\plots\\{}-{}\\Run{}\\fullSystematics\\'.format(bnb_type,mc_type,RUN)
elif DETSYS == True and DRAWSYS == False:
    SAVEDIR = 'C:\\Users\\Ryan\\python-workspace\\PELEE-newmcfilteredsamples\\plots\\{}-{}\\Run{}\\StatsandDetsys\\'.format(bnb_type,mc_type,RUN)
else:
    SAVEDIR = 'C:\\Users\\Ryan\\python-workspace\\PELEE-newmcfilteredsamples\\plots\\{}-{}\\Run{}\\justStats\\'.format(bnb_type,mc_type,RUN)
if 'fullsel' in tag.lower():
    SAVEDIR += 'fullsel\\'
elif 'presel' in tag.lower():
    SAVEDIR += 'presel\\'
if not os.path.exists(SAVEDIR): os.makedirs(SAVEDIR)
QUERY, track_cuts = 'nslice == 1', None
#plot_title, XTIT = moreFunctions.smart_title(tag, SELECTLONGEST, RUN, bnb_type, PRESEL, USECRT)
##################################################################
if not SELECTLONGEST:
    tag += '_alltracks'
if not DRAWDATA:
    tag += "_nodata"
if not DRAWRATIO:
    tag += '_noratio'

for KIND in KINDS:
    if SAVEFIG and len(VARIABLES) > 1:
        pdf = matplotlib.backends.backend_pdf.PdfPages(SAVEDIR+"combined_{}_{}_{}.pdf".format(tag,date_time,KIND))
    for (VARIABLE, BINS, RANGE, XTIT) in zip(VARIABLES, BIN, RANGES, XTITS):
        #####################################################################
        #get specific cuts based on what the tag is
        #QUERY, track_cuts = moreFunctions.get_Cuts(tag, ISRUN3)
        #######################################################
        # plotting
        weights, pot = moreFunctions.get_scaling(RUN, bnb_type, mc_type)
        my_plotter = plotter.Plotter(samples, weights, pot=pot)
        
        if DETSYS:
            if 'fullsel' in tag.lower():
                detsys = moreFunctions.get_Detsys('fullsel',VARIABLE,RANGE,BINS)
            elif 'presel' in tag.lower():
                detsys = moreFunctions.get_Detsys('presel',VARIABLE,RANGE,BINS)
        else:
            detsys = None
        
        fig, ax1, ax2 = my_plotter.plot_variable(
            VARIABLE,   
            query=QUERY,
            kind=KIND, #sample, interaction, backtracked_pdg
            track_cuts = track_cuts,
            select_longest = SELECTLONGEST, #this is true by default in self._selection
            title=XTIT,
            #bins=asymm_bins,
            bins=BINS,
            stacksort=4, #0-numerical, 1-weights, 2-eLee on top, 3-eLee+nue on top, 4-numu on top
            range=RANGE,
            ratio=DRAWRATIO,
            draw_sys=DRAWSYS,
            detsys=detsys,
            #draw_data=DRAWDATA,
        )[0:3]
        print("Profile likelihood: {} sigma @ {} POT".format(my_plotter.significance_likelihood,pot))
        print("s/sqrt(b): {} sigma @ {} POT".format(my_plotter.significance, pot))

        #ax1.set_ylim(0,40)
        #ax1.set_yscale("log")
        #ax1.set_ylim(0,12000)
        ax1.set_ylim(0,ax1.get_ylim()[1]*1.5)
        #ax2.set_ylim(0,2.5)
        #ax1.set_title(plot_title, fontsize=15)

        if SAVEFIG:
            fn = VARIABLE+"_"+date_time+"_"+tag+'_'+KIND
            fn += ".pdf"
            print("saving to {}...".format(SAVEDIR+fn))
            fig.tight_layout()
            fig.savefig(SAVEDIR+fn)
            if len(VARIABLES) > 1:
                pdf.savefig(fig)
        plt.show()
        
    if SAVEFIG and len(VARIABLES) > 1:
        pdf.close()

## How Shower-Like are our protons?

In [ ]:
importlib.reload(moreFunctions)
# CCQE_samples, CCQE_muon_samples, CCQE_proton_samples
#### PREPPING ####
MC_protons = CCQE_proton_samples['mc']
DATA_protons = CCQE_proton_samples['data']
for protons in [MC_protons,DATA_protons]:
    protons['trk_score'] = protons['trk_score_v'].apply(lambda x: x[0])
    protons['trk_energy_proton'] = protons['trk_energy_proton_v'].apply(lambda x: x[0])
    protons['trk_llr_pid_score'] = protons['trk_llr_pid_score_v'].apply(lambda x: x[0])

fig, axes = plt.subplots(2,1, figsize=(7,8))

##################################################
# PLOT 1
# Want to plot efficiency as function of proton E
# where efficiency = (track-like selected protons) / all selected protons
TRACK_CUT = 0.5 #everything > 0.5 is track, < 0.5 is shower
PID_CUT = -0.2
bin_edges = np.linspace(0,0.55, 8)
signal = 'trk_llr_pid_score < {}'.format(PID_CUT)

MC_proton_tracks = MC_protons.query('trk_score > {}'.format(TRACK_CUT))
bins,MC_bin_eff,MC_bin_err = moreFunctions.Eff(MC_protons, MC_proton_tracks, 'trk_energy_proton', signal, bin_edges) #get bin info

DATA_proton_tracks = DATA_protons.query('trk_score > {}'.format(TRACK_CUT))
_,DATA_bin_eff,DATA_bin_err = moreFunctions.Eff(DATA_protons, DATA_proton_tracks, 'trk_energy_proton', signal, bin_edges)

MC_bin_eff = list(MC_bin_eff)
MC_bin_err = list(MC_bin_err)
MC_bin_eff.append(MC_bin_eff[-1])
MC_bin_err.append(MC_bin_err[-1])

axes[0].step(bin_edges, 
         MC_bin_eff, 
         where='post', label='MC, track cut > {}'.format(TRACK_CUT))
axes[0].fill_between(
        bin_edges,
        np.array(MC_bin_eff)+np.array(MC_bin_err),
        np.array(MC_bin_eff)-np.array(MC_bin_err),
        alpha=0.3, step='post')
axes[0].errorbar(
        bins,DATA_bin_eff,
        yerr=DATA_bin_err, xerr=bins[0]-bin_edges[0],
        label='Data, track cut > {}'.format(TRACK_CUT)
)
        

axes[0].set_title(r"$\nu_{\mu}$ CC QE protons")
axes[0].set_ylabel("Track Fraction")
axes[0].set_xlabel("Range-Based Proton Track Energy [GeV]")
axes[0].legend(loc = 'lower center')

################################################
# PLOT 2
# just a scatter plot of the data used for the efficiency calculation
axes[1].scatter(MC_protons['trk_energy_proton'],
            MC_protons['trk_score'],
            s=1, label = 'MC')
axes[1].scatter(DATA_protons['trk_energy_proton'],
            DATA_protons['trk_score'],
            s=1, label = 'Data')

axes[1].plot([min(list(protons['trk_energy_proton'])),max(list(protons['trk_energy_proton']))],[TRACK_CUT,TRACK_CUT],'r--', label='Track Cut = {}'.format(TRACK_CUT))
axes[1].set_xlabel('Range-Based Proton Track Energy [GeV]')
axes[1].set_ylabel('Track Score')
axes[1].set_title(r"$\nu_{\mu}$ CC QE protons")
axes[1].legend()

fig.tight_layout()
fig.savefig('plots\\Run3\\0304_numupresel\\proton-studies\\track-fraction\\trackscore_protons.pdf')

#### Efficiency and Purity

In [ ]:
SAVEFIG = False
DRAWERRS = False
##############################################

eff_samples = ['mc']
pur_samples = ['mc','dirt','nue','ext']
signal = 'abs(nu_pdg) == 14 and ccnc == 0'
signal += ' and nmuon==1'
signal += ' and true_nu_vtx_x > {} and true_nu_vtx_x < {}'.format(FVx[0],FVx[1])
signal += ' and true_nu_vtx_y > {} and true_nu_vtx_y < {}'.format(FVy[0],FVy[1])
signal += ' and true_nu_vtx_z > {} and true_nu_vtx_z < {}'.format(FVz[0],FVz[1])
bin_edges = np.linspace(0.15,1.55,15)
df = samples['mc'] #unbiased denominator
df_cut = samples['mc'].query() #selected numerator
 
bins,bin_eff,bin_err = Eff(df,df_cut,'nu_e',signal,bin_edges) #get bin info
bin_eff, bin_err = list(bin_eff), list(bin_err)
overall_eff = np.sum(df_cut.query(signal).query('nu_e < 1.55')['nu_e']) / np.sum(df.query(signal).query('nu_e < 1.55')['nu_e'])
bin_eff.append(bin_eff[-1])
bin_err.append(bin_err[-1])
plt.step(bin_edges, 
         bin_eff, 
         where='post', label = "Efficiency: {}% overall".format(round(overall_eff*100,2)))
plt.fill_between(
        bin_edges,
        np.array(bin_eff)+np.array(bin_err),
        np.array(bin_eff)-np.array(bin_err),
        alpha=0.3, step='post')


bins,bin_pur,bin_err = Pur(fullsel_samples, weights, 'nu_e', signal, bin_edges)
bin_pur, bin_err = list(bin_pur), list(bin_err)
overall_pur = np.sum(df_cut.query(signal).query('nu_e > 0.15 and nu_e < 1.55')['nu_e']*weights['mc']) / (np.sum(df_cut.query('nu_e > 0.15 and nu_e < 1.55')['nu_e'])*weights['mc'] + np.sum(fullsel_samples['dirt'].query('nu_e > 0.15 and nu_e < 1.55')['nu_e'])*weights['dirt'] + np.sum(fullsel_samples['ext'].query('nu_e > 0.15 and nu_e < 1.55')['nu_e'])*weights['ext'] + np.sum(fullsel_samples['nue'].query('nu_e > 0.15 and nu_e < 1.55')['nu_e'])*weights['nue'])
bin_pur.append(bin_pur[-1])
bin_err.append(bin_err[-1])
plt.step(bin_edges, 
         bin_pur, 
         where='post', label = "Purity: {}% overall".format(round(overall_pur,4)*100))
if DRAWERRS:
    plt.fill_between(
            bin_edges,
            np.array(bin_pur)+np.array(bin_err),
            np.array(bin_pur)-np.array(bin_err),
            alpha=0.3, step='post')


plt.legend(fontsize=15)
plt.xlim([bin_edges[0],bin_edges[-1]])
plt.ylim([0,1])
plt.title("Efficiency and Purity")
plt.xlabel(r"True $\nu$ Energy [GeV]")
if SAVEFIG: plt.savefig('plots\\Run3\\0304\\pur-eff\\fullsel-presel_pur-eff.pdf')

### study mean/median CRT values

In [ ]:
#make hist
BINS = np.linspace(14250, 17750,100)
plot_samples = samples

medians = {}
runs = np.linspace(14300,17500, 35)
median_all, median_data, median_ext = [], [], []
mean_all, mean_data, mean_ext = [], [], []

for r,run in enumerate(runs[:-1]):
    query = "nslice == 1"
    query += ' and crthitpe > 0'
    r0 = run
    r1 = runs[r+1]
    query += ' and run >= {} and run < {}'.format(r0,r1)
    data_crts = plot_samples['data'].query(query)['crthitpe']
    ext_crts = plot_samples['ext'].query(query)['crthitpe']
    all_crts = list(data_crts) + list(ext_crts)
    '''
    if data_crts.size > 0:
        median_data.append(np.median(np.array(data_crts)))
    else:
        median_data.append(0)
    if ext_crts.size > 0:
        median_ext.append(np.median(np.array(ext_crts)))
    else:
        median_ext.append(0)
    if len(all_crts) > 0:
        median_all.append(np.median(np.array(all_crts)))
    else:
        median_all.append(0)
    '''
    median_data.append(np.median(np.array(data_crts)))
    median_ext.append(np.median(np.array(ext_crts)))
    median_all.append(np.median(np.array(all_crts)))
    mean_data.append(np.mean(np.array(data_crts)))
    mean_ext.append(np.mean(np.array(ext_crts)))
    mean_all.append(np.mean(np.array(all_crts)))
    
fig = plt.figure(figsize=(10,5))
plt.plot(runs[:-1],mean_all, 'k', label='mean combined')
plt.plot(runs[:-1],mean_data, 'r', label='mean data (1e20)')
plt.plot(runs[:-1],mean_ext, 'g', label='mean ext')
plt.plot(runs[:-1],median_all, 'k--', label='median combined')
plt.plot(runs[:-1],median_data, 'r--',label='median data (1e20)')
plt.plot(runs[:-1],median_ext, 'g--', label='median ext')
plt.plot([16300,16300],[0,225], 'b--', label='run 16300')

plt.xlabel('runs')
plt.ylabel('median CRT hit PE val')
plt.title("CRThitPE per ~100 runs, JustSliceID")
plt.legend(bbox_to_anchor=(1.04,1), loc='upper left')
plt.ylim([150,250])
plt.tight_layout()
plt.savefig(ls.plots_path+RUN+ls.SAMPLE+"\\CRThitPE-fixed\\CRTHITPE_vsrun_051120.pdf")

## Resolution Studies

In [ ]:
###########################################
## Is neutrino E calculated consistently?
###########################################

plot_sample = fullsel_samples
if SAMPLE_TYPE == 'G1':
    my_plotter = moreFunctions.get_plotter(tag, plot_sample, pot, True, ISRUN3)
else:
    my_plotter = moreFunctions.get_plotter(tag, plot_sample, pot, False, ISRUN3)

varname1, varname2 = 'nu_e', 'reco_nu_e_range_v'

QUERY, track_cuts = 'nslice == 1', None
QUERY += ' and ( (_opfilter_pe_beam > 0 and _opfilter_pe_veto < 20) or bnbdata == 1 or extdata == 1)'
#track_cuts = None

tag = "fullsel_samples" #tag gets added to end of .png name
SAVEFIG = False
SAVEDIR = ls.plots_path+RUN+ls.SAMPLE+'\\aprilcm-response\\'

if not os.path.exists(SAVEDIR):
    os.makedirs(SAVEDIR)

fig, axes = my_plotter.plot_2d(
                varname1,varname2,
                QUERY, track_cuts,
                range_x = (0.15,1.55),
                range_y = (0.15,1.55),
                bins_x = 50,
                bins_y = 50,
)

if SAVEFIG:
    fn = varname1+"-"+varname2+"_"+date_time+"_"+tag
    fn += ".pdf"
    print("saving to {}...".format(SAVEDIR+fn))
    fig.savefig(SAVEDIR+fn)
plt.show()

In [ ]:
######################################
## Make resolution plots
#######################################
importlib.reload(moreFunctions)
importlib.reload(plotter)

varx, vary = 'theta', 'trk_theta_v'
labelx, labely = r"True lepton $\theta$", r"Reco Muon Cand $\theta$"
labelx, labely = varx, vary
ranges = (0,np.pi)

varx, vary = 'nu_e', 'reco_nu_e_range_v'
labelx, labely = r"$E_{True}^{\nu_{\mu}}$ [GeV]", r"$E_{range}^{\nu_{\mu}}$ [GeV]"
ranges = (0.15,1.55)

#fig,axes = plt.subplots(1,2, figsize=(10,5), sharey=True)

tag = "fullsel_muon_samples_trueQE"
SAVEFIG = True
DRAWDIAG = True
DRAW2D = True
DRAW1D = False
range_z = None #[0,1]
BINS1D = None #np.linspace(-4,4,100)
SAVEDIR = ls.plots_path+RUN+ls.SAMPLE+'\\EB-response\\'
#####################################################################
if "fullsel_muon_sample" in tag.lower():
    plot_sample = fullsel_muon_samples #or samples, or presel_sapmles, 
    plot_title = "Fullsel"
elif "fullsel_ccqe_muon" in tag.lower():
    plot_sample = fullsel_CCQE_muon_samples 
    plot_title = "CCQE"
elif "fullsel_ccqe" in tag.lower():
    plot_sample = fullsel_CCQE_samples 
    plot_title = "CCQE"
elif "fullsel_nomcs_sample" in tag.lower():
    plot_sample = fullsel_noMCS_samples 
    plot_title = "Fullsel, No MCS Cut, "
elif "presel_contained_sample" in tag.lower():
    plot_sample = presel_contained_samples
    plot_title = "Presel, Contained Tracks, "
elif "samples" in tag.lower():
    plot_sample = samples
    if "presel" in ls.SAMPLE:
        plot_title = "Presel"
    else:
        plot_title = "NoSel"        
        
if "nopion" in tag.lower():
    plot_title += r', True o$\pi$'
if "trueqe" in tag.lower():
    plot_title += ', true QE INT type'
    
title_1d = plot_title
title_2d = plot_title

#get specific cuts based on what the tag is
QUERY, track_cuts = moreFunctions.get_Cuts(tag, ISRUN3)
    
if "interaction" in tag: kind = "interaction" 
else: kind = "event_category"

title = None
if 'nomcs' in tag.lower():
    title_1d += ' (no MCS cut)'
    title_2d += ' (no MCS cut)'
elif 'invertmcs' in tag.lower():
    title_1d += ' (inverted MCS cut)'
    title_2d += ' (inverted MCS cut)'
    
LOG2D = False
if 'log' in tag.lower():
    LOG2D = True
    
bins = 100
if 'lowres' in tag.lower():
    bins = 15
elif 'mediumres' in tag.lower():
    bins = 50
elif 'highres' in tag.lower():
    bins = 100
elif 'highestres' in tag.lower():
    bins = 250

if 'firstfew' in tag.lower():
    ranges = (0.15,0.55)
    title_1d += ' (first 550 MeV)'
    
KIND1D = "diffratio" # "diff" or "diffratio" or "ratio"
if 'ratio' in tag.lower():
    KIND1D = 'ratio'
elif 'diff' in tag.lower():
    KIND1D = 'diff'

if not os.path.exists(SAVEDIR): os.makedirs(SAVEDIR)
#######################################################
# plotting
if SAMPLE_TYPE == 'G1':
    my_plotter = moreFunctions.get_plotter(tag, plot_sample, pot, True, ISRUN3)
else:
    my_plotter = moreFunctions.get_plotter(tag, plot_sample, pot, False, ISRUN3)

labelx = moreFunctions.deJargoner(varx)
labely = moreFunctions.deJargoner(vary)

if DRAW2D:
    fig = plt.figure()
    axis = plt.gca()
    _,_,image = my_plotter.plot_2d_oneplot(varx,vary,
                    QUERY,
                    track_cuts = track_cuts,
                    range_x = ranges,
                    range_y = ranges,
                    bins_x = bins,
                    bins_y = bins,
                    #range_z = range_z,
                    figure = fig,
                    axis = axis,
                    log=LOG2D,
                  )

    if DRAWDIAG:
        ranges = np.array(ranges)
        ranges_down40 = ranges - 0.04
        axis.plot(ranges, ranges, 'r--', alpha=0.5)
    axis.set_xlabel(labelx)    
    axis.set_ylabel(labely)
    axis.set_title(title_2d)
    fig.colorbar(image, ax=axis)
    if SAVEFIG:
        fn = varx+"-"+vary+"_"+date_time+"_"+tag+"_2DRES"+'.pdf'
        print("saving to {}...".format(SAVEDIR+fn))
        plt.tight_layout()
        fig.savefig(SAVEDIR+fn)
    plt.show()

if DRAW1D:
    fig = plt.figure()
    axis = plt.gca()
    my_plotter.plot_1d_oneplot(
        varx,vary,
        query=QUERY,
        track_cuts=track_cuts,
        fig=fig,axis=axis,
        kind1d = KIND1D,
        bins=BINS1D)
    
    if KIND1D == 'diff':
        label = "{} - {}".format(labely, labelx)
    elif KIND1D == 'ratio':
        label = "{} / {}".format(labely, labelx)
    else:
        label = "({} - {}) / {}".format(labely, labelx, labely)
    axis.set_xlabel(label)
    axis.set_title(title_1d)
    if SAVEFIG:
        fn = varx+"-"+vary+"_"+date_time+"_"+tag+'_1DRES'+".pdf"
        print("saving to {}...".format(SAVEDIR+fn))
        plt.tight_layout()
        fig.savefig(SAVEDIR+fn)
    plt.show()


In [ ]:
######################################
## Make resolution plots
#######################################
importlib.reload(moreFunctions)
importlib.reload(plotter)
plot_sample = fullsel_samples
if SAMPLE_TYPE == 'G1':
    my_plotter = moreFunctions.get_plotter(tag, plot_sample, pot, True, ISRUN3)
else:
    my_plotter = moreFunctions.get_plotter(tag, plot_sample, pot, True, ISRUN3)
    
##############################################################
tag = 'fullsel_samples_highRes_firstfew'
SAVEFIG = True
MAKE1D = True
MAKE2D = False
SAVEDIR = ls.plots_path+RUN+ls.SAMPLE+'\\aprilcm-response\\'
####################


labelx = r"$E_{True}^{\nu_{\mu}}$"
labely = r"$E_{range}^{\nu_{\mu}}$ [GeV]"

#fig,axes = plt.subplots(1,2, figsize=(10,5), sharey=True)

QUERY = 'nslice==1'
track_cuts = None

if MAKE1D:
    fig = plt.figure()
    axes = plt.gca()
    
    
    
    print("making 1d plot...")
    

if MAKE2D:
    fig = plt.figure()
    axes = plt.gca()

    if 'mediumres' in tag.lower():
        bins = 50
    elif 'highres' in tag.lower():
        bins = 100
    elif 'highestres' in tag.lower():
        bins = 250

    ranges = (0.15,1.55)
    if 'firstfew' in tag.lower():
        ranges = (0.15,0.55)

    print("making 2d plot...")
    _,_,image = my_plotter.plot_2d_oneplot('nu_e','reco_nu_e_range_v',QUERY,
                    track_cuts = track_cuts,
                    range_x = ranges,
                    range_y = ranges,
                    bins_x = bins,
                    bins_y = bins,
                    range_z = [0,1],
                    figure = fig,
                    axis = axes,
                  )

    axes.set_xlabel(labelx)    
    axes.set_ylabel(labely)
    axes.set_title("Energy Resolution")
    fig.colorbar(image, ax=axes)

    if SAVEFIG:
        fn = varname1+"-"+varname2+"_"+date_time+"_"+tag
        fn += ".pdf"
        print("saving to {}...".format(SAVEDIR+fn))
        fig.savefig(SAVEDIR+fn)
    plt.show()